In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd.profiler import record_function

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.yolov8cls_path import Model

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu')
device=torch.device('cpu') 

In [5]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [6]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


# Inference at 640x640

In [43]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((640, 640)),
                                      transforms.Normalize(mean=means, std=stds)])

In [44]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [45]:
from torch.profiler import profile, ProfilerActivity

In [46]:
model = Model(num_classes=1000, 
              residual_connection=False, 
              CSP=False, 
              add_hidden=False,
              classifyV8=False,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [47]:
model.eval()
model.training

False

In [48]:
next(model.parameters()).device

device(type='cpu')

In [37]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-1/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total GFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.04%     835.600us       100.00%        2.165s     216.482ms

In [49]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-1/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.05%     110.200us       100.00%     212.642ms     212.642ms

# Inference at 224x224

In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [8]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [9]:
img = train_dataset[0][0].unsqueeze(0)

In [10]:
from torch.profiler import profile, ProfilerActivity

In [11]:
model = Model(num_classes=1000, 
              residual_connection=False, 
              CSP=False, 
              add_hidden=False,
              classifyV8=False,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [12]:
model.eval()
model.training

False

In [13]:
next(model.parameters()).device

device(type='cpu')

In [14]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-1/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(12):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.15%     792.300us       100.00%     519.982ms      51.998ms

In [20]:
img = train_dataset[6][0].unsqueeze(0).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-1/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i in range(3):
        # torch.cuda.synchronize(device=device)
        with record_function('inference'):
            with torch.no_grad():
                model(img)
        # torch.cuda.synchronize(device=device)
        prof.step()
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                   ProfilerStep*         0.17%      88.500us       100.00%      51.998ms      51.998ms

# GFLOPs at 224x224 using Ultralytics functions

In [19]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [22]:
get_flops(model, imgsz=224)

3.535195552

In [23]:
get_flops_with_torch_profiler(model, imgsz=224)

3.5143270400000004

# GFLOPs at 640x640 using Ultralytics functions

In [25]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [26]:
get_flops(model, imgsz=640)

28.8587392

In [27]:
get_flops_with_torch_profiler(model, imgsz=640)

28.688384

In [24]:
from evaluation import count_parameters, show_tree, generate_dataframe

# Count Parameters

In [28]:
count_parameters(model)

10528544